Title: MCQ TF FITB GENERATOR     
Author: Raikibul Hasan

**Table of contents**<a id='toc0_'></a>    
- [Preparation](#toc1_)    
    - [Install libraries](#toc1_1_1_)    
    - [Import libraries](#toc1_1_2_)    
    - [Load/Insert Article/file](#toc1_1_3_)    
- [MCQ GENERATE](#toc2_)    
    - [Extract keyword using PKE](#toc2_1_1_)    
    - [Split the article into an array/list of individual sentences](#toc2_1_2_)    
    - [Map the sentences which contain the keywords](#toc2_1_3_)    
    - [Get the sense of the word](#toc2_1_4_)    
    - [first distractor generate from WordNet](#toc2_1_5_)    
    - [Second distractor generator](#toc2_1_6_)    
    - [Find and map the distractors to the keywords](#toc2_1_7_)    
    - [Show generates MCQ](#toc2_1_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Preparation](#toc0_)
In this section, we will import the necessary libraries and dataset.

### <a id='toc1_1_1_'></a>[Install libraries](#toc0_)
***If you have installed  library once skip it otherwise make installed varibale false and run***

In [66]:
installed = True


In [67]:
def install_library():
    !pip install flashtext
    !pip install pywsd
    !pip install string
    !pip install nltk
    !pip install spacy
    !pip install git+https: // github.com/boudinfl/pke.git
    !pip install matplotlib
    !python - m spacy download en_core_web_sm
    !pip install ipywidgets


if not installed:
    install_library()


### <a id='toc1_1_2_'></a>[Import libraries](#toc0_)

In [68]:
import warnings
import ipywidgets as widgets
import os
import random
import re
import json
import requests
from nltk.corpus import wordnet as wn
from pywsd.lesk import cosine_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import adapted_lesk
from pywsd.similarity import max_similarity
from flashtext import KeywordProcessor
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import nltk
import pke
import string
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('popular')
# File system manangement
# Suppress warnings
warnings.filterwarnings('ignore')


[nltk_data] Downloading package stopwords to C:\Users\Raikibul
[nltk_data]     Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Raikibul
[nltk_data]     Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to C:\Users\Raikibul
[nltk_data]    |     Hasan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to C:\Users\Raikibul
[nltk_data]    |     Hasan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to C:\Users\Raikibul
[nltk_data]    |     Hasan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to C:\Users\Ra

### <a id='toc1_1_3_'></a>[Load/Insert Article/file](#toc0_)
For generating MCQ you need to load text file or you can also copy any article and paste here.    </br>
</br> 
***Choose an option for Load Article***


In [69]:
loadsetting = widgets.RadioButtons(
    options=['example', 'fileupload', 'instert-text'],
    value='example',  # Defaults to 'pineapple'
    # layout={'width': 'max-content'}, # If the items' names are long
    description='Choose a option for Load Article:',
    disabled=False
)
loadsetting


RadioButtons(description='Choose a option for Load Article:', options=('example', 'fileupload', 'instert-text'…

In [70]:
if loadsetting.value == 'example':

    file = open("stody.txt", "r")  # "r" deontes read version open
    text = file.read()

elif loadsetting.value == 'instert-text':
    # defining some widgets
    file = widgets.Textarea(value='',
                            description='Article/text', )
else:
    file = widgets.FileUpload(
        accept='',  # Accepted file extension e.g. '.txt',
        multiple=False  # True to accept multiple files upload else False
    )
    text = file.value
file


<_io.TextIOWrapper name='stody.txt' mode='r' encoding='cp1252'>

# <a id='toc2_'></a>[MCQ GENERATE](#toc0_)

### <a id='toc2_1_1_'></a>[Extract keyword using PKE](#toc0_)

In [71]:
def getImportantWords(art):
    # Using the Multipartite Unsupervised Extractor as our extractor
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=art, language='en')
    # We are only considering proper nouns as valid candidates for our keywords
    pos = {'PROPN'}
    # Stoplist contains the words to be avoided
    stops = list(string.punctuation)
    # These stand for the brackets as in lrb=left round bracket="(" and so on
    stops += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stops += stopwords.words('english')
    # Sets the candidate selection criteria, as in, which should be considered and which should be avoided
    extractor.candidate_selection(pos=pos)
    extractor.candidate_weighting()  # Sets the preference criteria for the candidates
    result = []
    # Gets the 25 best candidates according to the criteria set
    ex = extractor.get_n_best(n=25)
    for each in ex:
        result.append(each[0])
    return result


# Get the important words (keywords) from the text article using the above function
impWords = getImportantWords(text)
print(impWords)


['computer', 'ibm', 'bios', 'windows', 'interface manager', 'alu', 'minicomputers', 'latin', 'cpu', 'ms office', 'midrange computers', 'software', 'pdas', 'ram', 'fun facts', 'industrial revolution']


### <a id='toc2_1_2_'></a>[Split the article into an array/list of individual sentences](#toc0_)

In [72]:
def splitTextToSents(art):
    s = [sent_tokenize(art)]
    s = [y for x in s for y in x]
    # Removes all the sentences that have length less than 15 so that we can ensure that our questions have enough length for context
    s = [sent.strip() for sent in s if len(sent) > 15]
    return s


# Achieve a well splitted set of sentences from the text article
sents = splitTextToSents(text)
print(sents)


['The term computer was once used to refer to a person who did computation, unlike today.', 'The development of early prototypes that led to the modern computer is credited to many individuals throughout history.', 'A series of breakthroughs, beginning with transistor computers and then integrated circuit computers, resulted in the development of transistor technology and the integrated circuit chip, causing digital computers to largely replace analogue computers.', 'A computer is an electronic tool that manipulates data or information.', 'It can store, retrieve, and process information.', 'We can type documents, send emails, play games, and browse the Web using a computer.', 'It can also be used to edit spreadsheets, presentations, and even videos, or create them.', 'Early computers were conceived only as devices for calculating.', 'Simple manual devices such as the abacus have helped individuals do calculations since ancient times.', 'Some mechanical devices were built early in the I

### <a id='toc2_1_3_'></a>[Map the sentences which contain the keywords](#toc0_)

In [73]:
def mapSents(impWords, sents):
    # Using keyword processor as our processor for this task
    processor = KeywordProcessor()
    keySents = {}
    for word in impWords:
        keySents[word] = []
        processor.add_keyword(word)  # Adds key word to the processor
    for sent in sents:
        # Extract the keywords in the sentence
        found = processor.extract_keywords(sent)
        for each in found:
            # For each keyword found, map the sentence to the keyword
            keySents[each].append(sent)
    for key in keySents.keys():
        temp = keySents[key]
        # Sort the sentences according to their decreasing length in order to ensure the quality of question for the MCQ
        temp = sorted(temp, key=len, reverse=True)
        keySents[key] = temp
    return keySents


# Achieve the sentences that contain the keywords and map those sentences to the keywords using this function
mappedSents = mapSents(impWords, sents)
print(mappedSents)


{'computer': ['The control unit handles the various components of the computer; it reads and interprets (decodes) the instructions for the program, transforming them into control signals that activate other computer parts.', 'The control unit handles the various components of the computer; it reads and interprets (decodes) the instructions for the program, transforming them into control signals that activate other computer parts.', 'This is also supported by a lot of pieces of evidence where we can even see in daily life how the computer is not just present in an organization but is also available right in the pockets of everyone.', 'The hardware includes circuits, computer chips, graphics cards, sound cards, memory (RAM), motherboards, displays, power supplies, cables, keyboards, printers and "mice" input devices.', 'There are four main components of a general-purpose computer: the arithmetic logic unit (ALU), the control unit, the memory, and the I/O (collectively called input and ou

### <a id='toc2_1_4_'></a>[Get the sense of the word](#toc0_)

In [74]:
def getWordSense(sent, word):
    word = word.lower()
    if len(word.split()) > 0:  # Splits the word with underscores(_) instead of spaces if there are multiple words
        word = word.replace(" ", "_")
    synsets = wn.synsets(word, 'n')  # Sysnets from Google are invoked
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output = adapted_lesk(sent, word, pos='n')
        lowest_index = min(synsets.index(
            wup), synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

### <a id='toc2_1_5_'></a>[first distractor generate from WordNet](#toc0_)

In [75]:
def getDistractors(syn,word):
    dists=[]
    word=word.lower()
    actword=word
    if len(word.split())>0: #Splits the word with underscores(_) instead of spaces if there are multiple words
        word.replace(" ","_")
    hypernym = syn.hypernyms() #Gets the hypernyms of the word
    if len(hypernym)==0: #If there are no hypernyms for the current word, we simple return the empty list of distractors
        return dists
    for each in hypernym[0].hyponyms(): #Other wise we find the relevant hyponyms for the hypernyms
        name=each.lemmas()[0].name()
        if(name==actword):
            continue
        name=name.replace("_"," ")
        name=" ".join(w.capitalize() for w in name.split())
        if name is not None and name not in dists: #If the word is not already present in the list and is different from he actial word
            dists.append(name)
    return dists

### <a id='toc2_1_6_'></a>[Second distractor generator](#toc0_)

In [76]:
def getDistractors2(word):
    word=word.lower()
    actword=word
    if len(word.split())>0: #Splits the word with underscores(_) instead of spaces if there are multiple words
        word=word.replace(" ","_")
    dists=[]
    url= "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word) #To get ditractors from ConceptNet's API
    obj=requests.get(url).json()
    for edge in obj['edges']:
        link=edge['end']['term']
        url2="http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2=requests.get(url2).json()
        for edge in obj2['edges']:
            word2=edge['start']['label']
            if word2 not in dists and actword.lower() not in word2.lower(): #If the word is not already present in the list and is different from he actial word
                dists.append(word2)
    return dists

### <a id='toc2_1_7_'></a>[Find and map the distractors to the keywords](#toc0_)

In [77]:
mappedDists={}
for each in mappedSents:
    #print('sent',each)
    wordsense=getWordSense(mappedSents[each][0],each) #gets the sense of the word
    #print('wordsem',wordsense)
    if wordsense: #if the wordsense is not null/none
        dists=getDistractors(wordsense,each) #Gets the WordNet distractors
        if len(dists)==0: #If there are no WordNet distractors available for the current word
            dists=getDistractors2(each) #The gets the distractors from the ConceptNet API
        if len(dists)!=0: #If there are indeed distractors from WordNet available, then maps them
            mappedDists[each]=dists
    else: #If there is no wordsense, the directly searches/uses the ConceptNet
        dists=getDistractors2(each)
        if len(dists)>0: #If it gets the Distractors then maps them
            mappedDists[each]=dists
print(mappedDists)

{'computer': ['Assembly', 'Bagger', 'Calculator', 'Calender', 'Cash Machine', 'Comber', 'Concrete Mixer', 'Corker', 'Cotton Gin', 'Decoder', 'Farm Machine', 'Franking Machine', 'Hop-picker', 'Machine Tool', 'Machinery', 'Milking Machine', 'Motor', 'Pavior', 'Perpetual Motion Machine', 'Pile Driver', 'Power Shovel', 'Power Tool', 'Press', 'Printer', 'Record Player', 'Riveting Machine', 'Self-feeder', 'Simulator', 'Slicer', 'Slot Machine', 'Snow Thrower', 'Sorter', 'Stamp', 'Staple Gun', 'Stapler', 'Textile Machine', 'Time Machine', 'Trimmer', 'Workhorse', 'Zamboni'], 'windows': ['Windowpane'], 'minicomputers': ['Bulletin Board System', 'File Server', 'Mainframe', 'Minicomputer', 'Multiprocessor', 'Personal Computer', 'Von Neumann Machine', 'Workstation'], 'latin': ['Abator', 'Abjurer', 'Abomination', 'Abstainer', 'Achiever', 'Acquaintance', 'Acquirer', 'Active', 'Actor', 'Adjudicator', 'Admirer', 'Adoptee', 'Adult', 'Adventurer', 'Adversary', 'Advisee', 'Advocate', 'Affiant', 'African',

### <a id='toc2_1_8_'></a>[Show generates MCQ](#toc0_)

In [79]:
#Step 9- The final step is to present our MCQ in a nice and readable manner.
from contextlib import redirect_stdout

with open('out.pdf', 'w') as f:
    with redirect_stdout(f):
        print("**************************************        Multiple Choice Questions        *******************************")
        print()
        iterator = 1 #To keep the count of the questions
        for each in mappedDists:
            sent=mappedSents[each][0]
            p=re.compile(each,re.IGNORECASE) #Converts into regular expression for pattern matching
            op=p.sub("________",sent) #Replaces the keyword with underscores(blanks)
            print("Question %s-> "%(iterator),op) #Prints the question along with a question number
            options=[each.capitalize()]+mappedDists[each] #Capitalizes the options
            options=options[:4] #Selects only 4 options
            opts=['a','b','c','d']
            random.shuffle(options) #Shuffle the options so that order is not always same
            for i,ch in enumerate(options):
                print("\t",opts[i],") ", ch) #Print the options
            print()
            iterator+=1 #Increase the counter

In [81]:
!pip install fpdf

  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40722 sha256=b52cb03f3add2426a8ab13d8308865a1e08ef3106e9e6b620e7df1adb2f70cbb
  Stored in directory: c:\users\raikibul hasan\appdata\local\pip\cache\wheels\44\35\8b\86ce00cec7e4d13c5f189680ae0fa82f919bedc066c2cddae9
Successfully built fpdf


In [83]:
from fpdf import FPDF
  
# save FPDF() class into
# a variable pdf
pdf = FPDF()  
  
# Add a page
pdf.add_page()
  
# set style and size of font
# that you want in the pdf
pdf.set_font("Arial", size = 8)
 
# open the text file in read mode
f = open("out.txt", "r")
 
# insert the texts in pdf
for x in f:
    pdf.cell(200, 10, txt = x, ln = 1, align = 'L')
  
# save the pdf with name .pdf
pdf.output("mygfg.pdf") 

''